In [1]:
import numpy as np
import pandas as pd

In [2]:
import matplotlib.pyplot as plt
from matplotlib import style
import matplotlib.ticker as ticker
import seaborn as sns

In [3]:
from sklearn.datasets import load_boston
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score, make_scorer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid
from sklearn.inspection import permutation_importance
import multiprocessing

In [4]:
from xgboost import XGBClassifier

In [5]:
labels = pd.read_csv('../../csv/train_labels.csv')
labels.head()

,building_id,damage_grade
0,802906,3
1,28830,2
2,94947,3
3,590882,2
4,201944,3


In [6]:
values = pd.read_csv('../../csv/train_values.csv')
values.T

,0,1,2,3,4,5,6,7,8,9,...,260591,260592,260593,260594,260595,260596,260597,260598,260599,260600
building_id,802906,28830,94947,590882,201944,333020,728451,475515,441126,989500,...,560805,207683,226421,159555,827012,688636,669485,602512,151409,747594
geo_level_1_id,6,8,21,22,11,8,9,20,0,26,...,20,10,8,27,8,25,17,17,26,21
geo_level_2_id,487,900,363,418,131,558,475,323,757,886,...,368,1382,767,181,268,1335,715,51,39,9
geo_level_3_id,12198,2812,8973,10694,1488,6089,12066,12236,7219,994,...,5980,1903,8613,1537,4718,1621,2060,8163,1851,9101
count_floors_pre_eq,2,2,2,2,3,2,2,2,2,1,...,1,2,2,6,2,1,2,3,2,3
age,30,10,10,10,30,10,25,0,15,0,...,25,25,5,0,20,55,0,55,10,10
area_percentage,6,8,5,6,8,9,3,8,8,13,...,5,5,13,13,8,6,6,6,14,7
height_percentage,5,7,5,5,9,5,4,6,6,4,...,3,5,5,12,5,3,5,7,6,6
land_surface_condition,t,o,t,t,t,t,n,t,t,t,...,n,t,t,t,t,n,t,t,t,n
foundation_type,r,r,r,r,r,r,r,w,r,i,...,r,r,r,r,r,r,r,r,r,r


In [7]:
to_be_categorized = ["land_surface_condition", "foundation_type", "roof_type",\
                     "position", "ground_floor_type", "other_floor_type",\
                     "plan_configuration", "legal_ownership_status"]
for row in to_be_categorized:
    values[row] = values[row].astype("category")
values.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260601 entries, 0 to 260600
Data columns (total 39 columns):
 #   Column                                  Non-Null Count   Dtype   
---  ------                                  --------------   -----   
 0   building_id                             260601 non-null  int64   
 1   geo_level_1_id                          260601 non-null  int64   
 2   geo_level_2_id                          260601 non-null  int64   
 3   geo_level_3_id                          260601 non-null  int64   
 4   count_floors_pre_eq                     260601 non-null  int64   
 5   age                                     260601 non-null  int64   
 6   area_percentage                         260601 non-null  int64   
 7   height_percentage                       260601 non-null  int64   
 8   land_surface_condition                  260601 non-null  category
 9   foundation_type                         260601 non-null  category
 10  roof_type                       

In [8]:
datatypes = dict(values.dtypes)
for row in values.columns:
    if datatypes[row] != "int64" and datatypes[row] != "int32" and \
       datatypes[row] != "int16" and datatypes[row] != "int8":
        continue
    if values[row].nlargest(1).item() > 32767 and values[row].nlargest(1).item() < 2**31:
        values[row] = values[row].astype(np.int32)
    elif values[row].nlargest(1).item() > 127:
        values[row] = values[row].astype(np.int16)
    else:
        values[row] = values[row].astype(np.int8)

In [9]:
labels["building_id"] = labels["building_id"].astype(np.int32)
labels["damage_grade"] = labels["damage_grade"].astype(np.int8)
labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260601 entries, 0 to 260600
Data columns (total 2 columns):
 #   Column        Non-Null Count   Dtype
---  ------        --------------   -----
 0   building_id   260601 non-null  int32
 1   damage_grade  260601 non-null  int8 
dtypes: int32(1), int8(1)
memory usage: 1.2 MB


In [10]:
values['age_is_leq_than_100'] = (values['age'] <= 100).astype(np.int8)
# values['age_is_betw_100_and_200'] = ((values['age'] > 100) & (values['age'] <= 200)).astype(np.int8)
values['age_is_greater_than_200'] = (values['age'] > 200).astype(np.int8)
values[values['age'] >= 100]

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,age_is_leq_than_100,age_is_greater_than_200
216,612661,11,883,2822,5,100,12,11,t,r,...,0,0,0,0,0,0,0,0,1,0
324,737045,21,477,4348,3,190,6,7,t,r,...,0,0,0,0,0,0,0,0,0,0
386,435557,14,1120,3789,3,995,12,5,t,r,...,0,0,0,0,0,0,0,0,0,1
419,370360,6,1253,5667,4,105,11,9,t,r,...,0,0,0,0,0,0,0,0,0,0
472,393373,10,310,3637,2,100,4,4,t,r,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260093,34321,17,1393,5939,2,995,8,4,t,r,...,0,0,0,0,0,0,0,0,0,1
260469,1038559,3,1387,9059,5,105,9,11,t,r,...,0,0,0,0,0,0,0,0,0,0
260541,798100,13,1365,11849,1,100,6,3,t,r,...,0,0,0,0,0,0,0,0,1,0
260542,156434,8,696,7863,2,995,8,8,t,r,...,0,0,0,0,0,0,0,0,0,1


# Feature Engineering para XGBoost

In [11]:
important_values = values\
                .merge(labels, on="building_id")
important_values.drop(columns=["building_id"], inplace = True)
important_values["geo_level_1_id"] = important_values["geo_level_1_id"].astype("category")
important_values

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,age_is_leq_than_100,age_is_greater_than_200,damage_grade
0,6,487,12198,2,30,6,5,t,r,n,...,0,0,0,0,0,0,0,1,0,3
1,8,900,2812,2,10,8,7,o,r,n,...,0,0,0,0,0,0,0,1,0,2
2,21,363,8973,2,10,5,5,t,r,n,...,0,0,0,0,0,0,0,1,0,3
3,22,418,10694,2,10,6,5,t,r,n,...,0,0,0,0,0,0,0,1,0,2
4,11,131,1488,3,30,8,9,t,r,n,...,0,0,0,0,0,0,0,1,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260596,25,1335,1621,1,55,6,3,n,r,n,...,0,0,0,0,0,0,0,1,0,2
260597,17,715,2060,2,0,6,5,t,r,n,...,0,0,0,0,0,0,0,1,0,3
260598,17,51,8163,3,55,6,7,t,r,q,...,0,0,0,0,0,0,0,1,0,3
260599,26,39,1851,2,10,14,6,t,r,x,...,0,0,0,0,0,0,0,1,0,2


In [13]:

X_train, X_test, y_train, y_test = train_test_split(important_values.drop(columns = 'damage_grade'),
                                                    important_values['damage_grade'], test_size = 0.2, random_state = 123)

In [15]:
#OneHotEncoding
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    res = res.drop([feature_to_encode], axis=1)
    return(res) 

features_to_encode = ["geo_level_1_id", "land_surface_condition", "foundation_type", "roof_type",\
                     "position", "ground_floor_type", "other_floor_type",\
                     "plan_configuration", "legal_ownership_status"]
for feature in features_to_encode:
    X_train = encode_and_bind(X_train, feature)
    X_test = encode_and_bind(X_test, feature)

In [16]:
X_train

,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,...,plan_configuration_m,plan_configuration_n,plan_configuration_o,plan_configuration_q,plan_configuration_s,plan_configuration_u,legal_ownership_status_a,legal_ownership_status_r,legal_ownership_status_v,legal_ownership_status_w
260497,1001,6626,1,0,17,3,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
175563,675,4736,2,5,7,4,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
138080,744,9367,2,5,7,5,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
245465,516,9828,1,0,9,3,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
152380,0,9045,2,50,15,5,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192476,217,10644,1,25,4,6,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
17730,600,4813,2,20,13,8,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
28030,463,4692,2,10,9,4,1,1,0,0,...,0,0,0,0,0,0,0,0,1,0
15725,600,157,2,50,5,8,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0


In [17]:
import time

# min_child_weight = [0, 1, 2]
# max_delta_step = [0, 5, 10]

def my_grid_search():
    print(time.gmtime())
    i = 1
    df = pd.DataFrame({'subsample': [],
                       'gamma': [],
                       'learning_rate': [],
                       'max_depth': [],
                       'score': []})
    for subsample in [0.75, 0.885, 0.95]:
        for gamma in [0.75, 1, 1.25]:
            for learning_rate in [0.4375, 0.45, 0.4625]:
                 for max_depth in [5, 6, 7]:
                    model = XGBClassifier(n_estimators = 350,
                                          booster = 'gbtree',
                                          subsample = subsample,
                                          gamma = gamma,
                                          max_depth = max_depth,
                                          learning_rate = learning_rate,
                                          label_encoder = False,
                                          verbosity = 0)
                    model.fit(X_train, y_train)
                    y_preds = model.predict(X_test)
                    score = f1_score(y_test, y_preds, average = 'micro')
                    df = df.append(pd.Series(
                        data={'subsample': subsample,
                              'gamma': gamma,
                              'learning_rate': learning_rate,
                              'max_depth': max_depth,
                              'score': score},
                    name = i))
                    print(i, time.gmtime())
                    i += 1

    return df.sort_values('score', ascending = False)

current_df = my_grid_search()
df = pd.read_csv('grid-search/res-feature-engineering.csv')
df.append(current_df)
df.to_csv('grid-search/res-feature-engineering.csv')

current_df

time.struct_time(tm_year=2021, tm_mon=7, tm_mday=16, tm_hour=23, tm_min=14, tm_sec=27, tm_wday=4, tm_yday=197, tm_isdst=0)


/home/joaquinfontela/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


KeyboardInterrupt: 

In [51]:
import time

def my_grid_search():
    print(time.gmtime())
    i = 1
    df = pd.DataFrame({'subsample': [],
                       'gamma': [],
                       'learning_rate': [],
                       'max_depth': [],
                       'score': []})
    for subsample in [0.885]:
        for gamma in [1]:
            for learning_rate in [0.45]:
                for max_depth in [5,6,7,8]:
                    model = XGBClassifier(n_estimators = 350,
                                          booster = 'gbtree',
                                          subsample = subsample,
                                          gamma = gamma,
                                          max_depth = max_depth,
                                          learning_rate = learning_rate,
                                          label_encoder = False,
                                          verbosity = 0)
                    model.fit(X_train, y_train)
                    y_preds = model.predict(X_test)
                    score = f1_score(y_test, y_preds, average = 'micro')
                    df = df.append(pd.Series(
                        data={'subsample': subsample,
                              'gamma': gamma,
                              'learning_rate': learning_rate,
                              'max_depth': max_depth,
                              'score': score},
                    name = i))
                    print(i, time.gmtime())
                    i += 1

    return df.sort_values('score', ascending = False)

df = my_grid_search()
# df = pd.read_csv('grid-search/res-feature-engineering.csv')
# df.append(current_df)
df.to_csv('grid-search/res-feature-engineering.csv')

df

time.struct_time(tm_year=2021, tm_mon=7, tm_mday=13, tm_hour=21, tm_min=22, tm_sec=10, tm_wday=1, tm_yday=194, tm_isdst=0)


/home/joaquinfontela/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


1 time.struct_time(tm_year=2021, tm_mon=7, tm_mday=13, tm_hour=21, tm_min=24, tm_sec=23, tm_wday=1, tm_yday=194, tm_isdst=0)


/home/joaquinfontela/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


2 time.struct_time(tm_year=2021, tm_mon=7, tm_mday=13, tm_hour=21, tm_min=27, tm_sec=28, tm_wday=1, tm_yday=194, tm_isdst=0)


/home/joaquinfontela/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


3 time.struct_time(tm_year=2021, tm_mon=7, tm_mday=13, tm_hour=21, tm_min=31, tm_sec=41, tm_wday=1, tm_yday=194, tm_isdst=0)


/home/joaquinfontela/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


4 time.struct_time(tm_year=2021, tm_mon=7, tm_mday=13, tm_hour=21, tm_min=37, tm_sec=7, tm_wday=1, tm_yday=194, tm_isdst=0)


,subsample,gamma,learning_rate,max_depth,score
2,0.885,1.0,0.45,6.0,0.743577
3,0.885,1.0,0.45,7.0,0.742868
4,0.885,1.0,0.45,8.0,0.740968
1,0.885,1.0,0.45,5.0,0.740719


In [14]:
pd.read_csv('grid-search/res-no-feature-engineering.csv')\
    .nlargest(20, 'score')

FileNotFoundError: [Errno 2] No such file or directory: 'grid-search/res-no-feature-engineering.csv'

# Entreno tres de los mejores modelos con Voting.

In [18]:
xgb_model_1 = XGBClassifier(n_estimators = 350,
                          subsample = 0.885,
                          booster = 'gbtree',
                          gamma = 1,
                          learning_rate = 0.45,
                          label_encoder = False,
                          verbosity = 2)

xgb_model_2 = XGBClassifier(n_estimators = 350,
                          subsample = 0.950,
                          booster = 'gbtree',
                          gamma = 0.5,
                          learning_rate = 0.45,
                          label_encoder = False,
                          verbosity = 2)

xgb_model_3 = XGBClassifier(n_estimators = 350,
                          subsample = 0.750,
                          booster = 'gbtree',
                          gamma = 1,
                          learning_rate = 0.45,
                          label_encoder = False,
                          verbosity = 2)

xgb_model_4 = XGBClassifier(n_estimators = 350,
                          subsample = 0.80,
                          booster = 'gbtree',
                          gamma = 1,
                          learning_rate = 0.55,
                          label_encoder = False,
                          verbosity = 2)

In [19]:
rf_model_1 = RandomForestClassifier(n_estimators = 150,
                                  max_depth = None,
                                  max_features = 45,
                                  min_samples_split = 15,
                                  min_samples_leaf = 1,
                                  criterion = "gini",
                                  verbose=True)

rf_model_2 = RandomForestClassifier(n_estimators = 250,
                                  max_depth = None,
                                  max_features = 45,
                                  min_samples_split = 15,
                                  min_samples_leaf = 1,
                                  criterion = "gini",
                                  verbose=True,
                                  n_jobs =-1)

In [20]:
import lightgbm as lgb
lgbm_model_1 = lgb.LGBMClassifier(boosting_type='gbdt', 
                         colsample_bytree=1.0,
                         importance_type='split', 
                         learning_rate=0.15,
                         max_depth=None,
                         n_estimators=1600, 
                         n_jobs=-1,
                         objective=None,
                         subsample=1.0, 
                         subsample_for_bin=200000, 
                         subsample_freq=0)

lgbm_model_2 = lgb.LGBMClassifier(boosting_type='gbdt', 
                         colsample_bytree=1.0,
                         importance_type='split', 
                         learning_rate=0.15,
                         max_depth=25,
                         n_estimators=1750, 
                         n_jobs=-1,
                         objective=None,
                         subsample=0.7, 
                         subsample_for_bin=240000, 
                         subsample_freq=0)

lgbm_model_3 = lgb.LGBMClassifier(boosting_type='gbdt', 
                         colsample_bytree=1.0,
                         importance_type='split', 
                         learning_rate=0.20,
                         max_depth=40,
                         n_estimators=1450, 
                         n_jobs=-1,
                         objective=None,
                         subsample=0.7, 
                         subsample_for_bin=160000, 
                         subsample_freq=0)

In [21]:
import sklearn as sk
import sklearn.neural_network
neuronal_1 = sk.neural_network.MLPClassifier(solver='adam', 
                                     activation = 'relu',
                                     learning_rate_init=0.001,
                                     learning_rate = 'adaptive',
                                     verbose=True,
                                     batch_size = 'auto')

In [22]:
gb_model_1 = GradientBoostingClassifier(n_estimators = 300,
                                  max_depth = 9,
                                  min_samples_split = 2,
                                  min_samples_leaf = 3,
                                  subsample=0.6,
                                  verbose=True,
                                  learning_rate=0.15)

In [23]:
vc_model = VotingClassifier(estimators = [('xgb1', xgb_model_1),
                                          ('xgb2', xgb_model_2),
                                          ('rfm1', rf_model_1),
                                          ('lgbm1', lgbm_model_1),
                                          ('lgbm2', lgbm_model_2),
                                          ('gbm1', gb_model_1)],
                            voting = 'soft',
                            verbose = True)

In [24]:
vc_model.fit(X_train, y_train)

/home/joaquinfontela/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:14:45] WARNING: ../src/learner.cc:573: 
Parameters: { "label_encoder" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[20:14:45] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 124 extra nodes, 2 pruned nodes, max_depth=6
[20:14:45] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 116 extra nodes, 4 pruned nodes, max_depth=6
[20:14:46] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 120 extra nodes, 4 pruned nodes, max_depth=6
[20:14:46] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[20:14:46] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 1

[20:14:59] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 96 extra nodes, 10 pruned nodes, max_depth=6
[20:14:59] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 76 extra nodes, 22 pruned nodes, max_depth=6
[20:14:59] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 92 extra nodes, 8 pruned nodes, max_depth=6
[20:14:59] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 116 extra nodes, 2 pruned nodes, max_depth=6
[20:14:59] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 68 extra nodes, 8 pruned nodes, max_depth=6
[20:14:59] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 96 extra nodes, 10 pruned nodes, max_depth=6
[20:15:00] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 114 extra nodes, 8 pruned nodes, max_depth=6
[20:15:00] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 90 extra nodes, 14 pruned nodes, max_depth=6
[20:15:00] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 80 extra nodes, 18 pr

[20:15:12] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 100 extra nodes, 10 pruned nodes, max_depth=6
[20:15:12] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 114 extra nodes, 4 pruned nodes, max_depth=6
[20:15:12] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 98 extra nodes, 14 pruned nodes, max_depth=6
[20:15:12] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 96 extra nodes, 4 pruned nodes, max_depth=6
[20:15:13] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 90 extra nodes, 10 pruned nodes, max_depth=6
[20:15:13] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 124 extra nodes, 2 pruned nodes, max_depth=6
[20:15:13] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 98 extra nodes, 16 pruned nodes, max_depth=6
[20:15:13] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 44 extra nodes, 18 pruned nodes, max_depth=6
[20:15:13] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 84 extra nodes, 14 

[20:15:25] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 100 extra nodes, 12 pruned nodes, max_depth=6
[20:15:26] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 112 extra nodes, 8 pruned nodes, max_depth=6
[20:15:26] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 88 extra nodes, 16 pruned nodes, max_depth=6
[20:15:26] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 76 extra nodes, 10 pruned nodes, max_depth=6
[20:15:26] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 58 extra nodes, 6 pruned nodes, max_depth=6
[20:15:26] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 92 extra nodes, 22 pruned nodes, max_depth=6
[20:15:27] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 68 extra nodes, 14 pruned nodes, max_depth=6
[20:15:27] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 32 extra nodes, 16 pruned nodes, max_depth=6
[20:15:27] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 114 extra nodes, 6 

...

[20:21:26] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 76 extra nodes, 10 pruned nodes, max_depth=6
[20:21:26] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 80 extra nodes, 18 pruned nodes, max_depth=6
[20:21:26] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 88 extra nodes, 12 pruned nodes, max_depth=6
[20:21:26] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 80 extra nodes, 4 pruned nodes, max_depth=6
[20:21:26] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 86 extra nodes, 8 pruned nodes, max_depth=6
[20:21:27] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 80 extra nodes, 6 pruned nodes, max_depth=6
[20:21:27] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 94 extra nodes, 14 pruned nodes, max_depth=6
[20:21:27] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 76 extra nodes, 16 pruned nodes, max_depth=6
[20:21:27] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 88 extra nodes, 6 prun

[20:21:40] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 52 extra nodes, 2 pruned nodes, max_depth=6
[20:21:40] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 72 extra nodes, 14 pruned nodes, max_depth=6
[20:21:40] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 76 extra nodes, 12 pruned nodes, max_depth=6
[20:21:40] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 50 extra nodes, 6 pruned nodes, max_depth=6
[Voting] ..................... (2 of 6) Processing xgb2, total= 3.4min


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed:  3.0min finished


[Voting] ..................... (3 of 6) Processing rfm1, total= 3.0min
[Voting] .................... (4 of 6) Processing lgbm1, total=  25.3s
[Voting] .................... (5 of 6) Processing lgbm2, total=  30.7s
      Iter       Train Loss      OOB Improve   Remaining Time 
         1           0.8415           0.0689           16.41m
         2           0.7982           0.0418           16.27m
         3           0.7690           0.0292           16.15m
         4           0.7447           0.0213           16.11m
         5           0.7266           0.0169           16.07m
         6           0.7122           0.0129           16.04m
         7           0.7002           0.0102           15.99m
         8           0.6900           0.0087           15.95m
         9           0.6813           0.0073           15.93m
        10           0.6732           0.0060           15.86m
        20           0.6267           0.0025           15.38m
        30           0.6000           0.00

VotingClassifier(estimators=[('xgb1',
                              XGBClassifier(base_score=None, booster='gbtree',
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, gamma=1,
                                            gpu_id=None, importance_type='gain',
                                            interaction_constraints=None,
                                            label_encoder=False,
                                            learning_rate=0.45,
                                            max_delta_step=None, max_depth=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraint...
                             ('lgbm1',
                              LGBMClassifier(learning_rate=0.15, max_depth=None,
                                        

In [25]:
y_preds = vc_model.predict(X_test)
f1_score(y_test, y_preds, average='micro')

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed:    0.0s finished


0.5

In [26]:
test_values = pd.read_csv('../../csv/test_values.csv', index_col = "building_id")
test_values

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
building_id,,,,,,,,,,,,,,,,,,,,,
300051,17,596,11307,3,20,7,6,t,r,n,...,0,0,0,0,0,0,0,0,0,0
99355,6,141,11987,2,25,13,5,t,r,n,...,1,0,0,0,0,0,0,0,0,0
890251,22,19,10044,2,5,4,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
745817,26,39,633,1,0,19,3,t,r,x,...,0,0,1,0,0,0,0,0,0,0
421793,17,289,7970,3,15,8,7,t,r,q,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310028,4,605,3623,3,70,20,6,t,r,q,...,1,0,0,0,0,0,0,0,0,0
663567,10,1407,11907,3,25,6,7,n,r,n,...,0,0,0,0,0,0,0,0,0,0
1049160,22,1136,7712,1,50,3,3,t,r,n,...,0,0,0,0,0,0,0,0,0,0


In [27]:
test_values_subset = test_values
test_values_subset["geo_level_1_id"] = test_values_subset["geo_level_1_id"].astype("category")
test_values_subset

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
building_id,,,,,,,,,,,,,,,,,,,,,
300051,17,596,11307,3,20,7,6,t,r,n,...,0,0,0,0,0,0,0,0,0,0
99355,6,141,11987,2,25,13,5,t,r,n,...,1,0,0,0,0,0,0,0,0,0
890251,22,19,10044,2,5,4,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
745817,26,39,633,1,0,19,3,t,r,x,...,0,0,1,0,0,0,0,0,0,0
421793,17,289,7970,3,15,8,7,t,r,q,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310028,4,605,3623,3,70,20,6,t,r,q,...,1,0,0,0,0,0,0,0,0,0
663567,10,1407,11907,3,25,6,7,n,r,n,...,0,0,0,0,0,0,0,0,0,0
1049160,22,1136,7712,1,50,3,3,t,r,n,...,0,0,0,0,0,0,0,0,0,0


In [28]:
test_values_subset.shape

(86868, 38)

In [29]:
test_values_subset['age_is_leq_than_100'] = (test_values_subset['age'] <= 100).astype(np.int8)
# test_values_subset['age_is_betw_100_and_200'] = ((test_values_subset['age'] > 100) & (test_values_subset['age'] <= 200)).astype(np.int8)
test_values_subset['age_is_greater_than_200'] = (test_values_subset['age'] > 200).astype(np.int8)
test_values_subset[test_values_subset['age'] >= 100]

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,age_is_leq_than_100,age_is_greater_than_200
building_id,,,,,,,,,,,,,,,,,,,,,
3781,27,181,3563,5,100,3,10,t,r,n,...,0,0,0,0,0,0,0,0,1,0
666855,14,1120,3789,2,995,11,4,t,r,n,...,0,0,0,0,0,0,0,0,0,1
823756,15,874,7477,3,100,12,7,t,r,n,...,0,0,0,0,0,0,0,0,1,0
974381,22,1398,3765,2,995,4,5,t,r,n,...,0,0,0,0,0,0,0,0,0,1
118979,7,714,7850,2,100,4,5,t,r,n,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196137,11,131,3126,4,110,11,8,t,r,q,...,0,0,0,0,0,0,0,0,0,0
783720,27,181,2607,2,995,2,6,t,r,n,...,0,0,0,0,0,0,0,0,0,1
880161,13,480,1917,2,995,10,4,t,r,n,...,0,0,0,0,0,0,0,0,0,1


In [30]:
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    res = res.drop([feature_to_encode], axis=1)
    return(res) 

features_to_encode = ["geo_level_1_id", "land_surface_condition", "foundation_type", "roof_type",\
                     "position", "ground_floor_type", "other_floor_type",\
                     "plan_configuration", "legal_ownership_status"]
for feature in features_to_encode:
    test_values_subset = encode_and_bind(test_values_subset, feature)
test_values_subset

,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,...,plan_configuration_m,plan_configuration_n,plan_configuration_o,plan_configuration_q,plan_configuration_s,plan_configuration_u,legal_ownership_status_a,legal_ownership_status_r,legal_ownership_status_v,legal_ownership_status_w
building_id,,,,,,,,,,,,,,,,,,,,,
300051,596,11307,3,20,7,6,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
99355,141,11987,2,25,13,5,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
890251,19,10044,2,5,4,5,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
745817,39,633,1,0,19,3,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
421793,289,7970,3,15,8,7,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310028,605,3623,3,70,20,6,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
663567,1407,11907,3,25,6,7,1,1,1,0,...,0,0,0,0,0,0,0,0,1,0
1049160,1136,7712,1,50,3,3,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0


In [31]:
# Genero las predicciones para los test.
preds = vc_model.predict(test_values_subset)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed:    4.0s finished


In [32]:
submission_format = pd.read_csv('../../csv/submission_format.csv', index_col = "building_id")

In [33]:
my_submission = pd.DataFrame(data=preds,
                             columns=submission_format.columns,
                             index=submission_format.index)

In [34]:
my_submission.head()

,damage_grade
building_id,
300051,3
99355,2
890251,2
745817,1
421793,3


In [35]:
my_submission.to_csv('../../csv/predictions/jf/vote/jf-model-7-submission.csv')

In [36]:
!head ../../csv/predictions/jf/vote/jf-model-7-submission.csv

building_id,damage_grade
300051,3
99355,2
890251,2
745817,1
421793,3
871976,2
691228,1
896100,3
343471,2
